# Set up workspace

In [1]:
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import numpy as np

import sys
sys.path.append('/home/jovyan/normalizing-cdr-accounting/scripts')

from process_raw_hector_output import *
from condense_hector_output import *

# Load model output

In [2]:
fpath = '//home/jovyan/normalizing-cdr-accounting/data/hector-output/'
fname = 'output_DAC_emissionsDriven/outputstream_DAC_emissionsDriven.csv'
DAC_emissionsDriven=process_csv_to_xarray(fpath+fname, startyear=1750, filetype='C++')

fname = 'output_DOR_emissionsDriven/outputstream_DOR_emissionsDriven.csv'
DOR_emissionsDriven=process_csv_to_xarray(fpath+fname, startyear=1750, filetype='C++')

fname = 'output_DOR_concentrationDriven/outputstream_DOR_concentrationDriven.csv'
DOR_concentrationDriven=process_csv_to_xarray(fpath+fname, startyear=1750, filetype='C++')

fname = 'output_DOR_concentrationDriven_highWind/outputstream_DOR_concentrationDriven_highWind.csv'
DOR_concentrationDriven_highWind=process_csv_to_xarray(fpath+fname, startyear=1750, filetype='C++')

fname = 'output_DOR_concentrationDriven_lowWind/outputstream_DOR_concentrationDriven_lowWind.csv'
DOR_concentrationDriven_lowWind=process_csv_to_xarray(fpath+fname, startyear=1750, filetype='C++')

# Postprocess data

In [3]:
DAC_emissionsDriven_short = condense_output(df=DAC_emissionsDriven,
                                            deployment_year=1745+2000)

DOR_emissionsDriven_short = condense_output(df=DOR_emissionsDriven,
                                            deployment_year=1745+2000+1)

DOR_concentrationDriven_short = condense_output(df=DOR_concentrationDriven,
                                                deployment_year=1745+2000+1,
                                                concentration_driven=True)

DOR_concentrationDriven_highWind_short = condense_output(df=DOR_concentrationDriven_highWind,
                                                deployment_year=1745+2000+1,
                                                concentration_driven=True)

DOR_concentrationDriven_lowWind_short = condense_output(df=DOR_concentrationDriven_lowWind,
                                                deployment_year=1745+2000+1,
                                                concentration_driven=True)

In [4]:
# Adjust for time step discrepancy between DOR and DAC
last_year=DOR_emissionsDriven_short.years_after_deployment[-1]
DAC_emissionsDriven_short=DAC_emissionsDriven_short.where(DAC_emissionsDriven_short.years_after_deployment<=last_year,drop=True)

### Make synthetic data for concentration-driven DAC and emission
(not necessary to simulate)

In [5]:
# Make empty data array for DAC impact on CO2
DAC_concentrationDriven_empty = xr.DataArray(
    data=np.zeros(np.shape(DAC_emissionsDriven_short.delta_atmos_co2_GtCO2)),
    dims=["year"],
    coords=dict(
        year=DAC_emissionsDriven_short.year)
)
DAC_concentrationDriven=DAC_concentrationDriven_empty.to_dataset(name='implied_delta_atmos_co2_GtCO2')

DAC_concentrationDriven['years_after_deployment']=DAC_emissionsDriven_short.years_after_deployment
DAC_concentrationDriven['implied_delta_atmos_co2_GtCO2']=(DAC_concentrationDriven['implied_delta_atmos_co2_GtCO2'].where(DAC_emissionsDriven_short.years_after_deployment<=0,-1))

In [6]:
# Make empty data array for emission impact on CO2

da_emission_concentrationDriven = xr.DataArray(
    data=-DAC_concentrationDriven['implied_delta_atmos_co2_GtCO2'],
    dims=["year"],
    coords=dict(
        year=DAC_concentrationDriven.year)
)
emission_concentrationDriven=da_emission_concentrationDriven.to_dataset(name='implied_delta_atmos_co2_GtCO2')
emission_concentrationDriven['years_after_deployment']=DAC_concentrationDriven['years_after_deployment']

# Save postprocessed data

In [7]:
output_path='/home/jovyan/normalizing-cdr-accounting/data/postprocessed-hector/netcdfs/'
DAC_emissionsDriven_short.to_netcdf(output_path+'DAC_emissionsDriven_postprocessed.nc')
DOR_emissionsDriven_short.to_netcdf(output_path+'DOR_emissionsDriven_postprocessed.nc')
DOR_concentrationDriven_short.to_netcdf(output_path+'DOR_concentrationDriven_postprocessed.nc')
DOR_concentrationDriven_highWind_short.to_netcdf(output_path+'DOR_concentrationDriven_highWind_postprocessed.nc')
DOR_concentrationDriven_lowWind_short.to_netcdf(output_path+'DOR_concentrationDriven_lowWind_postprocessed.nc')

In [8]:
emission_concentrationDriven.to_netcdf(output_path+'emission_concentrationDriven_synthetic_data.nc')
DAC_concentrationDriven.to_netcdf(output_path+'DAC_concentrationDriven_synthetic_data.nc')